In [59]:
import torch

torch.mps.empty_cache()  # MPS 백엔드의 GPU 캐시 비우기


In [60]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Mistral-7B 모델 불러오기
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# 모델이 잘 불러와졌는지 확인
print("Mistral-7B 모델이 성공적으로 로드되었습니다.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


Mistral-7B 모델이 성공적으로 로드되었습니다.


In [2]:
!pip install sentence-transformers faiss-cpu

In [73]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
import json
import numpy as np

# 📌 책 데이터 로드
file_path = "filtered_novels.json"
with open(file_path, "r", encoding="utf-8") as f:
    books = json.load(f)

# ✅ 새로운 임베딩 모델 적용 (더 정확한 검색 성능)
embedding_model = HuggingFaceEmbedding("BAAI/bge-m3")

# ✅ 실제 벡터 차원 확인
test_text = "테스트 문장입니다."
test_embedding = embedding_model.get_text_embedding(test_text)
dimension = np.array(test_embedding).shape[-1]  # 🔹 실제 차원 자동 설정

print(f"✅ 모델 차원: {dimension}")

# 📌 `\n` 제거 후 문서를 `Document` 객체로 변환
from llama_index.core.schema import Document
documents = [
    Document(
        text=book["description"].replace("\n", " ").strip(),
        metadata={"title": book["title"], "author": book["author"]}
    )
    for book in books if book["description"].strip()
]

# 📌 FAISS 벡터 저장소 생성 (자동 차원 적용)
faiss_index = faiss.IndexFlatL2(dimension)
vector_store = FaissVectorStore(faiss_index=faiss_index)

# 📌 StorageContext 활용
storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore(),
    vector_store=vector_store
)

# 📌 인덱스 생성
index = VectorStoreIndex.from_documents(documents, embed_model=embedding_model, storage_context=storage_context)

# 📌 인덱스 저장
index.storage_context.persist("index_data")
print(f"✅ FAISS 벡터 DB가 {dimension}차원으로 성공적으로 저장되었습니다.")




✅ 모델 차원: 1024


KeyError: 'author'

In [4]:
%pip install llama-index-vector-stores-faiss

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install llama-index-vector-stores-faiss faiss-cpu

In [72]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# ✅ 더 적합한 문장 임베딩 모델 사용 (스토리 검색 성능 향상)
embedding_model = HuggingFaceEmbedding("BAAI/bge-m3")  # 🔹 더 강력한 모델 적용

# 📌 저장된 FAISS 벡터 DB 로드
retriever = VectorIndexRetriever(index=index)

# 📌 유저 입력 예제
user_query = "수학교사, 알리바이"

# 📌 FAISS 벡터 검색 실행 (top_k=20 설정)
top_k = 20  # 🔹 최대 20개의 검색 결과 반환
retrieved_nodes = retriever.retrieve(user_query)[:top_k]  # 🔹 상위 k개 선택

# 📌 🔹 검색된 전체 결과 개수 확인
print(f"🔹 검색된 전체 책 개수: {len(retrieved_nodes)}")

# 📌 유사도 점수 0.3 이상인 결과만 필터링 (더 많은 책 포함)
filtered_nodes = [node for node in retrieved_nodes if node.score > 0.3]

# 📌 `description`이 없는 경우에도 책을 보여주도록 개선
relevant_books = []
for node in filtered_nodes:
    title = node.metadata.get("title", "알 수 없음")
    author = node.metadata.get("author", "알 수 없음")
    description = node.text.strip()

    # 🔹 `description`이 없어도 제목과 저자는 보여줌
    book_info = f"제목: {title}, 저자: {author}"
    if description:
        book_info += f"\n줄거리: {description}"

    relevant_books.append(book_info)

# 📌 검색된 책이 없는 경우 처리
if not relevant_books:
    relevant_books.append("검색 결과 없음")

# 📌 검색된 책 목록 출력
print("\n🔹 [추천된 책 목록] 🔹")
for i, book in enumerate(relevant_books, 1):
    print(f"🔹 후보 {i}\n{book}\n")



🔹 검색된 전체 책 개수: 2

🔹 [추천된 책 목록] 🔹
🔹 후보 1
제목: 섬, 짓하다 (김재희 장편소설), 저자: 김재희
줄거리: 치밀한 고증과 연민 어린 감성으로 무장한 「프로파일러 김성호 시리즈」 첫 번째 이야기 『섬, 짓하다』. 성형수술을 했다는 이유로 한 여성이 잔인하게 살해되고 주도적으로 여성을 비방해온 남학생이 용의자로 검거되지만 경찰청 소속 프로파일러 성호는 범행 가능성이 희박하다고 판단한다. 성호의 심문 후 용의자가 자살을 기도하고 이 일에 대한 문책으로 그는 수사에서 제외, 삼보섬에서 일어난 연쇄실종사건의 진원을 맡으며 전출된다. 섬의 음울한 분위기에 중압감을 느끼면서 본격적인 프로파일링에 착수한 성호는 실종된 자의 혼을 달래기 위한 씻김굿 현장에서 충격적인 일을 경험하는데…….

🔹 후보 2
제목: 십자도 살인사건, 저자: 윤자영
줄거리: # 살인이 일어나는 죽음의 수학여행에서 범인을 추리해 살아남아라 # 학교에서 배운 과학지식을 활용하는 국내 유일의 본격 과학 추리 소설 # 추리소설의 백미 클로즈드 서클(갇힌 상황)에서 진범을 추리하는 지적 희열을 느낀다  즐거운 수학여행을 기대하며 떠난 ‘십자도’라는 섬에서 벌어지는 살인극.  외부와 완벽히 차단된 섬과 그곳에 갇힌 23명의 학생들과 교사, 그리고 마을 주민들. 누가, 왜 이런 살인을 저지르는지 점점 미궁으로 빠지는 가운데 사건 해결을 위해 영재와 부회장인 민선, 그리고 부담임인 이지현 선생은 용의자로 청년회장과 담임 교사 고민환 선생을 주시한다. 한편, 이 학교의 문제아이자 중학생 때부터 각종 사건사고를 저질러온 장희종과 패거리들은 수학여행지인 ‘십자도’에서조차 선생님의 주의와 경고를 무시하고, 가지 말라는 곳에서 술을 마시고 담배를 피우는 등 재미를 위해 선생님의 말쯤은 깡그리 무시한다.  결국 범인에게 붙잡힌 장희종과 패거리들은 전기고문을 받아가며 자신들이 저질러온 범죄를 기억해 내야 했다.   과연, 장희종과 패거리들은 왜 범인의 타깃이 되었으며, 그들은 자신의 과오를 진심으로

In [ ]:
##캐쉬 삭제하기 🚮
from transformers.utils import cached_path

# Hugging Face 캐시 경로 확인
cache_dir = cached_path("")
print(f"📂 Hugging Face 캐시 경로: {cache_dir}")

# 전체 캐시 삭제
import shutil
shutil.rmtree(cache_dir)
print("✅ Hugging Face 캐시 삭제 완료!")
